## In this notebook I try CatBoostClassifier Model




### Import all needed Libraries

In [9]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from catboost import CatBoostClassifier
from sklearn.metrics import log_loss
from sklearn.model_selection import GridSearchCV
from sklearn import metrics

<a></a>
##  Get the Data

In [10]:
train = pd.read_csv('train_cleaned.csv')
test = pd.read_csv('test_cleaned.csv')
sample = pd.read_csv('sample_submission (2).csv')

## Feature Selection

In [11]:
cat_cols = ['Artist Name', 'Track Name']

In [12]:
cont_cols =[
 'Popularity',
 'danceability',
 'energy',
 'key',
 'loudness',
 'mode',
 'speechiness',
 'acousticness',
 'instrumentalness',
 'liveness',
 'valence',
 'tempo',
 'duration_in min/ms',
 'time_signature']

In [13]:
X = train[cat_cols+cont_cols]
y = train.Class
test2= test[cat_cols+cont_cols]

In [14]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.1)

## Train Model

In [15]:
arr = []

model = CatBoostClassifier(n_estimators=10000, random_state=10, eval_metric= 'MultiClass', cat_features=cat_cols)
model.fit(X_train, y_train,eval_set=[(X_test, y_test)], early_stopping_rounds=30, verbose=500)

print(log_loss(y_test, model.predict_proba(X_test)))
arr.append(log_loss(y_test, model.predict_proba(X_test)))


Learning rate set to 0.04716
0:	learn: 2.3064862	test: 2.3097152	best: 2.3097152 (0)	total: 841ms	remaining: 2h 20m 13s
500:	learn: 0.8024053	test: 0.8808473	best: 0.8808473 (500)	total: 8m 1s	remaining: 2h 32m 15s
Stopped by overfitting detector  (30 iterations wait)

bestTest = 0.870795043
bestIteration = 746

Shrink model to first 747 iterations.
0.8707950430050257


In [16]:
y_pred = model.predict_proba(X_test)

In [17]:
out_y_pred = pd.DataFrame(y_pred)
out_y_pred_vlaue = out_y_pred.idxmax(axis=1)

In [18]:
out_y_pred_vlaue.shape

(1440,)

In [19]:

confusion_matrix = metrics.confusion_matrix(y_test, out_y_pred_vlaue)

In [20]:
confusion_matrix

array([[ 33,   0,   0,   1,   3,   0,   0,   2,   0,   1,   2],
       [  0,  36,   5,   0,   0,   1,  31,   0,   4,   4,  34],
       [  0,   1,  70,   0,   0,   0,   8,   0,   1,  10,  24],
       [  3,   0,   0,  22,   1,   1,   0,   0,   0,   1,   0],
       [  4,   0,   0,   0,  25,   0,   0,   0,   0,   1,   0],
       [  0,   0,   2,   0,   0,  78,   2,   0,   0,   9,   1],
       [  0,   2,   8,   0,   0,   4, 124,   0,   2,   9,  61],
       [  0,   0,   0,   0,   0,   0,   0,  45,   0,   0,   0],
       [  0,   1,   1,   0,   0,   0,   4,   0, 127,   1,  29],
       [  0,   6,   5,   0,   1,   7,  20,   0,   4, 137,  24],
       [  0,  10,  16,   0,   0,   3,  31,   0,  21,  11, 305]],
      dtype=int64)

## Prediction

In [21]:
pred = model.predict_proba(test2)

In [22]:
out = pd.DataFrame(pred)
maxValues = out.idxmax(axis=1)

In [23]:
sample['Class'] = maxValues
sample.to_csv('catboost_out.csv', index=False)

### After preprossing data and train different models we find that CatBoosting Classifier is the best algorithm to gain less Error ( score = 0.69)